In [ ]:
from Bio.SeqUtils import MeltingTemp
from Bio.Seq import Seq

In [ ]:
def Tm(primerSeq):
    TM=MeltingTemp.Tm_NN(primerSeq,Mg=2.0,nn_table=MeltingTemp.DNA_NN4)
    return(TM)

In [ ]:
def seqWalk(inputSeq,delLength,start=0,end=0):
    if end==0:
        end=len(inputSeq)-1
    positions=range(int(len(inputSeq)/delLength))
    positionsIter=[i*delLength for i in positions if (i >= start/3 and i <= end/3)]
    return(positionsIter)

def windowSizing(position,delLength,inputSeq,tmDes,windowSize=8):
    #reversePrimer (left)
    inputSeq= inputSeq.upper()
    TM=0.0
    while TM<tmDes:
        primerR_for=inputSeq[(position-windowSize-2):position-2]
        TM=Tm(primerR_for)
        windowSize+=1
        forSeq = Seq(primerR_for)
        primerR = str(forSeq.reverse_complement())
        
    TM_r=TM #final Tm of reverse annealing region.
    
    #forwardPrimer (right)
    TM=0.0
    while TM<=tmDes:
        primerF=inputSeq[(position+delLength-2):(windowSize+position-2)]
        TM=Tm(primerF)
        windowSize+=1
    
    TM_f=TM #final Tm of forward annealing region.
    
    #overlap
    TM=0.0
    overlap=5
    while TM<tmDes+3:
        Fflank=primerR_for[(len(primerR_for)-overlap):len(primerR_for)]
        delPrimerF=(Fflank.lower()+primerF)
        
        Rflank=primerF[0:overlap]
        delPrimerR_for=primerR_for+Rflank.lower()
        
        forSeq = Seq(delPrimerR_for)
        delPrimerR = str(forSeq.reverse_complement())
        overlap+=1
        
        TM=Tm(delPrimerF[0:2*(overlap)])
        
    TM_OL=TM #final Tm of overlap.
    
    print("Deletion at pos: "+str(position)+" to "+str(position+delLength-1))
    print("Forward Primer: "+delPrimerF+"\tLength: "+str(len(primerF))+"\tTm: "+str(TM_f))
    print("Reverse Primer: "+delPrimerR+"\tLength: "+str(len(primerR))+"\tTm: "+str(TM_r))
    print("TM_OL: "+str(TM_OL))
    
    outputString="del_"+str(position)+"-"+str(position+delLength-1)+"_F,"+delPrimerF+","+str(round(TM_f,2))+"\n"+"del_"+str(position)+"-"+str(position+delLength-1)+"_R, "+delPrimerR+","+str(round(TM_r,2))+"\n"#string for text file (e.g. benchling input)
    return(outputString)



In [ ]:
inputSequence="ttaaaacagcctgtgggttgcacccacccacagggcccactgggcgctagcactctggtactgaggtacctttgtgcgcctgtttttactccccttcccccgaagtaacttagaagctgtaaatcaacgatcaatagcaggtgtggcacaccagtcataccttgatcaagcacttctgtttccccggactgagtatcaataggctgctcgcgcggctgaaggagaaaacgttcgttacccgaccaactacttcgagaagcttagtaccaccatgaacgaggcagggtgtttcgctcagcacaaccccagtgtagatcaggctgatgagtcactgcaacccccatgggcgaccatggcagtggctgcgttggcggcctgcccatggagaaatccatgggacgctctaattctgacatggtgtgaagagcctattgagctagctggtagtcctccggcccctgaatgcggctaatcctaactgcggagcacatgctcacaaaccagtgggtggtgtgtcgtaacgggcaactctgcagcggaaccgactactttgggtgtccgtgtttccttttattcctatattggctgcttatggtgacaatcaaagagttgttaccatatagctattggattggccatccggtgtgcaacagggcaattgtttacctatttattggttttgtaccattatcactgaagtctgtgatcactctcaaattcattttgaccctcaacacaatcaaacatgggctcacaagtgtccacacaacgctccggttcacacgaaaactctaactcagctaccgagggttccactataaactatactaccattaattactataaagattcctatgccgccacagcaggtaagcagagccttaagcaggacccagacaagtttgcaaatcctgtcaaagacatcttcactgaaatggcagcgccattaaaatctccatctgctgaggcatgtggttacagcgatcgggtggcacaattaactattggcaattctaccatcactacgcaagaagcagcaaacatcatagttggctatggtgagtggccttcctactgttcggactctgatgctactgcagtggacaaaccaacgcgcccagatgtttcggtgaataggttttacacattggacacaaaattgtgggagaaatcatccaaggggtggtactggaaattcccggatgtgttaactgaaaccggggtctttggtcaaaatgcacagttccactacctctatcggtcagggttctgcattcacgtgcagtgcaatgctagtaagttccaccaaggagcactcctagtcgctgtcctcccagagtatgtcattgggacagtggcaggtggcacagggacggaggatagccaccccccttataagcagactcaacccggtgctgatggcttcgaattgcaacacccgtacgtgcttgatgctggcattccaatatcacaattaacagtgtgcccacatcagtggattaatttgaggaccaacaattgtgccacaataatagtgccgtacataaacgcactaccctttgattctgccttgaaccattgtaactttggtctgctggttgtgcctattagcccgttagattatgaccaaggtgcgacgccagtgatccccattactatcactttggccccaatgtgttctgaatttgcaggccttagacaagcagttacgcaagggtttcctactgagctgaaacctggcacaaaccaatttttaaccactgacgatggcgtctcagcacccattctgccaaactttcaccccaccccgtgtatccatatacccggtgaagttagaaacttgctagagctatgccaggtggagaccattttagaggtcaacaatgtacctacgaatgccactagcttaatggagagactgcgcttcccggtctcagctcaagccgggaaaggtgagctatgtgcagtgttcagagctgaccctggacgaagtgggccatggcagtccaccttgttgggccagttgtgcgggtactacacccaatggtcaggatcactggaagtcaccttcatgttcaccgggtcctttatggctaccggcaagatgctcatagcatacacaccaccaggaggccccttacccaaggaccgggcgaccgccatgttgggcacgcacgtcatctgggactttgggctgcaatcgtctgtcacccttgtaataccatggatcagcaacactcattacagagcgcacgctcgagatggtgtgttcgactactacactacaggtttggttagcatatggtaccagacgaattatgtggttccaattggggcacccaatacagcctatataatagcattggcggcagcccagaagaacttcaccatgaagttgtgtaaggatgctagtgatatcctacagacaggcactatccagggagatagggtggcagatgtgattgagagttctataggggacagtgtgagcagagccctcacccgagctctaccggcacctaccggccaagacacacaggtaagcagccaccgattagatactggtaaagttccagcactccaagccgctgaaattggagcatcatcaaatgctagtgatgagagtatgattgagacacggtgtgttcttaattcacatagtacagctgagaccactcttgatagcttcttcagcagagcaggattagttggagagatagacctccctcttgaaggcacaaccaacccgaatgggtacgcaaactgggacatagacataacaggttacgcgcaaatgcgtagaaaggtggagctgttcacctacatgcgttttgacgcagagttcacctttgttgcatgcacccctaccgggcaagttgtcccgcaattgctccaatacatgtttgtaccacccggagcccccaagccagactccagagaatctctcgcatggcaaactgccactaatccctcagtttttgtgaagctgtcagaccccccagcacaggtttctgttccattcatgtcacctgcgagcgcctatcaatggttttatgacgggtatcccacattcggtgaacacaaacaggagaaagaccttgaatacggggcatgcccaaacaacatgatgggtacgttctcagtgcggactgtaggcacctcgaagtccaagtacccattggtgatcaggatttacatgaggatgaagcacgtcagggcgtggatacctcgcccaatgcgtaaccagaactatctattcaaagccaacccaaattatgctggtaattttattaaaccaactggtgccagtcgcacagcaatcaccaccctcgggaaatttggacagcagtccggagctatctacgtgggcaactttagagtggttacagacgcagtgtcaagagaagttgaagcactgaaaagtcacttgatcggctcagagggtgccgtggagaagattctaaagaacttagttaaactcatctctgcgctcgtcatcgtcatcaggagtgattatgacatggtcacattgacggcaacacttgccctgatcgggtgccacgggagcccttgggcctgggttaagtcgaagacagcatcaatcttgggcataccgatggctcagaagcagagtgcctcttggttaaagaagttcaacgatgcggcgagtgccgcgaaggggcttgagtggatctccaacaaaatcagtaaatttatcgattggctcaaggagaaaatcataccggctgctaaagagaaagtcgagtttctaaacaatctaaagcaactccccttattggagaaccaaatttctaatctcgaacagtcagcagcttcgcaggaggaccttgaggcgatgtttggcaacgtgtcttatctggcccacttctgccgcaaattccaacccctctatgccacggaagcaaagagggtgtacgccctagaaaagagaatgaataattacatgcagttcaagagcaaacaccgtattgaacctgtatgcctaatcatcagaggctcgcctggtactgggaagtccttggcaacagggattattgctagagccatagcagacaagtaccactccagtgtgtattccttacctccagacccagaccactttgacggatacaaacaacagatcgtcactgttatggacgacctatgccaaaacccagacgggaaagacatgtcactattttgtcagatggtctccacagtggattttataccgcctatggcatctctggaggagaagggagtctcattcacctccaagtttgtgattgcctccactaacgccagtaacatcatagtgccaacagtctcggattcagatgccattcgtcgccggttctttatggactgcgatattgaggtgaccgattcctataagacagagctgggcagacttgatgcagggagagcagccaggctgtgctctgagaacaacactgcaaactttaaacggtgcagtccattagtctgtgggaaagcaatccagcttagggataggaagtccaaggtgagatacagtgtggacacggtagtgagtgaacttatcagggagtataacaacagatcagttattgggaacaccattgaagctcttttccaaggaccccctaaatttagaccaataaggattagcttagaggagaagcccgcacctgatgctattagtgacttattagctagtgttgatagtgaagaggttcgccaatactgtagagatcagggatggattgtacctgattctcccaccaacgttgagcgccacttgaatagagctgtcttgattatgcagtctgtagccaccgtggtagcagttgtgtcccttgtttacgtcatctacaagttgttcgccggttttcaaggagcatattccggcgcccccaagcaaacactcaagaaaccagtgctgcgcacggcaactgtgcaggggccgagcttggacttcgccctatctctacttaggaggaacattaggcaggtccaaaccgaccagggccactttacaatgttaggagtgcgagaccgcttggctgtgctccccagacactcccaaccaggaaagaccatctgggttgaacacaaattagtgaagatcgtagatgctgtggagttagtagacgaacaaggggttaacttagagctcacactggtaacgcttgatactaacgaaaaatttagagacatcacaagattcataccagaaacaattagtcctgctagtgatgccactttagttataaatactgaacatatgcccagtatgtttgtgccagttggagatgtggtccagtatgggtttttgaaccttagtggtaagcccactcacaggactatgatgtacaatttcccaacaaaagcaggacagtgtggtggtgttgtgactgccgtgggtaaagtgattgggatccacattggtggcaacggtaggcaaggtttctgcgctgccctgaagaggggatacttttgcagtgaacaaggtgagatccaatggatgaagcccaacaaagaaactggcaggttgaacatcaacggacctactcgcactaagcttgaaccaagtgtctttcacgatgtgttcgaaggcactaaagagccagcagtgctgactagtaaagacccaaggctggaagttgactttgaacaggctcttttttcaaaatacgtggggaacacgcttcatgaacccgacgagtttgtcaaggaggcggccttacattatgccaaccaactcaagcagttagatatcaagaccaccaagatgagcatggaggatgcatgttacggcacagagaacctggaagctatagatcttcacacaagtgcaggatatccatacagtgcactaggcatcaagaaaaaggacattttggatccaacaactcgcgatgtcagcaagatgaaattctacatggacaagtatgggttggatctaccgtactctacttatgttaaagatgaacttagggccatcgacaagatcaagaaagggaagtctcgtctcatagaagcgagcagtctaaatgactcagtgtacttgagaatgacatttgggcacctttatgaagctttccacgccaacccaggtacaatcactggttcagctgttgggtgtaacccagatgtgttctggagcaagttaccaattctacttccaggatcgcttttcgcgtttgactactcggggtatgacgctagtctcagcccagtgtggttcagggcgctggagatagtcctgcgggaaattggatactccgaagacgcagtgtctctcatagaagggatcaatcacacccatcatgtgtaccgcaataaaacttattgtgttcttgggggaatgccctcaggttgctcaggcacctccattttcaactcgatgatcaacaatatcattattagaacactcctgattaaaacattcaaagggatagatctagatgaactgaacatggtggcctacggggatgatgtgttggctagttaccccttcccaattgactgtctggagttggcaagaacaggcaaggagtatggtctaactatgacccctgccgacaagtcaccctgctttaatgaggttacatgggagaatgccactttcttgaagagaggattcttgcctgatcatcaattcccgtttctcatccaccctacgatgccaatgagggagattcacgaatccattcgttggaccaaagatgcacgaagtactcaagatcacgtgcgctccctctgcttattagcatggcacaacgggaaagaggagtatgaaaaatttgtgagtgcaatcagatcagttccaattggaaaagcattggctataccaaattatgagaatctgagaagaaattggctcgaattgttttaaatttacagtttgtaactgaaccccaccagtaatctggtcgcgttaatgactggtgggggtaaatttgttataaccagaatagcaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"

In [ ]:
delLength=3
tmDes=52
windowSize=14
posArray=seqWalk(inputSequence,delLength,start=748,end=6879)
#print(posArray)

OF="EV-71_delPrimer_output.txt"
with open(OF,"w") as outfile:
    outfile.write("name,sequence,Tm\n")#header
    for i in posArray:
        outputString=windowSizing(i, delLength, inputSequence, tmDes, windowSize)
        outfile.write(outputString)
        

In [ ]:
delLength=3
tmDes=52
windowSize=14
posArray=seqWalk(inputSequence,delLength,start=748,end=6879)
#print(posArray)

OF="delPrimer_output.txt"

with open(OF,"w") as outfile:
    for i in posArray:
        outputString=windowSizing(i, delLength, inputSequence, tmDes, windowSize)
        outfile.write(outputString)
        

In [ ]:
delLength=3
tmDes=52
windowSize=14
posArray=seqWalk(inputSequence,delLength,start=748,end=6879)
#print(posArray)

OF="delPrimer_output.txt"

with open(OF,"w") as outfile:
    for i in posArray:
        outputString=windowSizing(i, delLength, inputSequence, tmDes, windowSize)
        outfile.write(outputString)
        